In [2]:
import numpy as np
from myprocessor import myprocessor
from plotter import plotter
import matplotlib.pyplot as plt

#Loads in the noise model from an npz file
NoiseModelFileName = 'NoiseModel_Full_11012023.npz'

myarray = np.load(NoiseModelFileName)

PSDarray = myarray['PSD']
rCorr = myarray['rCorr']
CorrBand = myarray['CorrBand']
BothLive = myarray['BothLive']
KurtMasking = myarray['KurtMasking']
TotalEvents = myarray['TotalEvents']

In [2]:
#PSDarray -> reshape -> multiply induction and collection planes by corresponding scale factors
#reshape back to linear
#Store in a new npz in same format
#Look into TH1D format? Otherwise send to Wenqiang with documentation and see if that works for him

Conv_Factor = np.zeros(plotter.numplanes)
Slope = np.array([3039,3039,3030]) #2 induction planes, 1 collection plane

Conv_Factor = 1/Slope * 24*10**3 * 1.6*10**(-4) * 14
print(Conv_Factor)

SortedPSDArray = plotter.convert_to_sorted(PSDarray)

N = myprocessor.minwvfm
T = myprocessor.SampleSpacing*N

for iTPC in range(2):
    for iPlane in range(3):
            SortedPSDArray[iTPC][iPlane] /= (2*T/N**2) #Divide out normalization
            SortedPSDArray[iTPC][iPlane] = np.sqrt(SortedPSDArray[iTPC][iPlane]) #Square root to get |FFT|
            SortedPSDArray[iTPC][iPlane] *= Conv_Factor[iPlane] #Get in units of mV

[0.01769003 0.01769003 0.01774257]


In [3]:
#Use this direct ADC->mV conversion using the dynamic range of ICEBERG

New_Conv_Factor = 1400/2**14
SortedPSDArray = plotter.convert_to_sorted(PSDarray)

N = myprocessor.minwvfm
T = myprocessor.SampleSpacing*N

for iTPC in range(2):
    for iPlane in range(3):
            SortedPSDArray[iTPC][iPlane] /= (2*T/N**2) #Divide out normalization
            SortedPSDArray[iTPC][iPlane] = np.sqrt(SortedPSDArray[iTPC][iPlane]) #Square root to get |FFT|
            SortedPSDArray[iTPC][iPlane] *= New_Conv_Factor #Get in units of mV


In [4]:
Converted_PSDArray = plotter.convert_to_mega(SortedPSDArray)

In [5]:
#Add the FFT from FEMB 8 to where FEMBs 6 and 9 would go
Converted_PSDArray[240:280] = Converted_PSDArray[280:320] #FEMB 9 u1 = FEMB 8 u1
Converted_PSDArray[360:400] = Converted_PSDArray[280:320] #FEMB 6 u1 = FEMB 8 u1
Converted_PSDArray[600:640] = Converted_PSDArray[680:720] #FEMB 6 v1 = FEMB 8 v1
Converted_PSDArray[720:760] = Converted_PSDArray[680:720] #FEMB 9 v1 = FEMB 8 v1
Converted_PSDArray[1088:1136] = Converted_PSDArray[1136:1184] #FEMB 9 z1 = FEMB 8 z1
Converted_PSDArray[1232:1280] = Converted_PSDArray[1136:1184] #FEMB 6 z1 = FEMB 8 z1

In [6]:
#Create new noise model file with conversion complete. Rejoice. Document. Send to Wenqiang.
savefile = "NoiseModel_mV_03_05_2024.npz"
np.savez(savefile,FFT = Converted_PSDArray, rCorr = rCorr, CorrBand = CorrBand, BothLive = BothLive, KurtMasking = KurtMasking, TotalEvents = TotalEvents)

In [5]:
print(Converted_PSDArray[3][4])
print(PSDarray[3][4])
print(Converted_PSDArray[3][4]/PSDarray[3][4])

28.488978346496
0.0012187791238614223
23375.01339556523


In [8]:
freq = np.fft.rfftfreq(plotter.minwvfm,plotter.SampleSpacing)
print(plotter.minwvfm)
print(plotter.SampleSpacing)
print(plotter.minwvfm*plotter.SampleSpacing)

2128
5e-07
0.001064


In [9]:
#Just the relevant information for Wenqiang:
savefile = "ICEBERG_Noise_Model_For_Wirecell"
np.savez(savefile,FFT = Converted_PSDArray,freq = freq, N = plotter.minwvfm, T = plotter.minwvfm*plotter.SampleSpacing)
